In [1]:
# CS6510 Assignment 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
from collections import defaultdict
import math
import copy

In [2]:
myname = "Prabhjot Singh"

In [3]:
class DecisionTree():

    def __init__(self):
        self.use_entropy = False

    def learn_with_calculate_entropy_and_gini_index(self, training_set, title):

        self.all_attributes = title.copy()
        self.types_of_attributes = {k:'numeric' for k in self.all_attributes.keys()}


        self.list_of_vertex = {}
        self.data=[]
        self.tree_prune = True
        for k in range(10,20):

            self.use_gini_index = False

            self.n_tile = k

            for N in xrange(0, 10):

                training_data = [x for i, x in enumerate(training_set) if (i-N) % 10 != 0]
                cross_validation_data = [x for i, x in enumerate(training_set) if (i-N) % 10 == 0]

                self.list_of_vertex.clear()
                del self.data[:]
                self.data = training_data

                self.train( training_data , title )

                if self.tree_prune:
                    self.prune(cross_validation_data)
                    
                accuracy = self.calculate_classific_accuracy(cross_validation_data)

                print("training accuracy=%.4f" % accuracy)

                f = open(myname+"result.txt", "w")
                f.write("accuracy: %.4f" % accuracy)
                f.close()

        f.close()

    def classify(self, classif_data):
        
        vertex_id = 'root_vertex'
        self.major_type_output = False
        result = 0

        while vertex_id:

            vertex = self.list_of_vertex[vertex_id]

            if vertex['type'] == 'bottom':
                return vertex['large_value']

            attrib = vertex['column_selected']
            tval = classif_data[self.all_attributes[attrib]]
            vertex_id=None
            if self.types_of_attributes[attrib] == 'numeric':
                prev=0.0
                
                dictionaries = {self.list_of_vertex[Key]['bin']:Key for Key in vertex['child_vertexs']}
                all_the_keys = dictionaries.keys()
                all_the_keys.sort()

                for data_value in all_the_keys:
                    new_key = dictionaries[data_value]
                    if prev < tval <= data_value:
                        vertex_id = new_key
                        break
                    prev=data_value

        return result

    def generate_n_dimensions(self, K, set_data_data_range,attribute_vertexs,result_attributes):

        length_of_data = len(set_data_data_range)
        dall_data_values={}
        all_data_values=[]

        for i in set_data_data_range:
            g = self.data[i][self.all_attributes[attribute_vertexs]]

            if g not in dall_data_values:
                dall_data_values[g]=[i]
            else:
                dall_data_values[g].append(i)

            all_data_values.append(g)
        all_data_values.sort()

        set_of_results = {}

        if length_of_data <= 2:

            data_all_data_values = dall_data_values.keys()
            data_all_data_values.sort()

            for all_values in data_all_data_values:

                if all_values not in set_of_results:
                    set_of_results[all_values] = {}
                    set_of_results[all_values]['bin_records'] = []
                    set_of_results[all_values]['bin_records_result_count'] = {}

                set_of_results[all_values]['bin_records']=dall_data_values[all_values]


        else:
            if (length_of_data < 2*K):
                new_key = length_of_data/2
            else:
                new_key = K

            dividend = length_of_data / new_key
            rem = length_of_data % new_key
            previous_value = 0

            for i in range(1,new_key+1):

                ki = i*dividend-1
                if (i <= rem):
                    ki += i
                else:
                    ki += rem

                if all_data_values[ki] not in set_of_results:
                    set_of_results[all_data_values[ki]] = {}
                    set_of_results[all_data_values[ki]]['bin_records'] = []
                    set_of_results[all_data_values[ki]]['bin_records_result_count'] = {}

                for all_values in all_data_values[previous_value:ki+1]:
                    if all_values in dall_data_values:
                        set_of_results[all_data_values[ki]]['bin_records'].extend(dall_data_values[all_values])

                        for v in dall_data_values[all_values]:
                            g = self.data[v][self.all_attributes[result_attributes]]
                            if g in set_of_results[all_data_values[ki]]['bin_records_result_count']:
                                set_of_results[all_data_values[ki]]['bin_records_result_count'][g] += 1
                            else:
                                set_of_results[all_data_values[ki]]['bin_records_result_count'][g] = 1

                        del dall_data_values[all_values]
                previous_value = ki+1

        return set_of_results


    def train ( self, training_data, title):

        self.end_target = 'quality'
        self.list_of_vertex['root_vertex']={ 'key':'root_vertex', 'parent':None, 'type' : 'root_vertex', 'all_attributes':title.copy(), 'set_data_data_range': set(range(len(training_data))) , 'child_vertexs': [], 'bin':None, 'parentAttrib':None, 'height':0, 'column_selected':None, 'parentKey':None }

        list_stack = []
        list_stack.append('root_vertex')
        self.edge_entropy_gain = False
        while list_stack:

            vertex_id = list_stack.pop(0)

            vertex = self.list_of_vertex[vertex_id]

            lv = [self.data[i][self.all_attributes[self.end_target]] for i in vertex['set_data_data_range']]

            if len(lv):
                large_value = max(lv,key=lv.count)
                vertex['large_value']=large_value

            vertex['total']=len(vertex['set_data_data_range'])

            errCount=0
            for i in vertex['set_data_data_range']:
                if self.data[i][self.all_attributes[self.end_target]] != large_value:
                    errCount +=1

            vertex['error_for_majority']=errCount


            if ( len(vertex['all_attributes']) == 1):
                vertex['type']='bottom'
                continue
            else:

                entropy_value1 = calculate_entropy_and_gini_index(self,vertex['set_data_data_range'],self.end_target,self.end_target)

                if ( entropy_value1 == 0 ):
                    vertex['type']='bottom'
                    continue
                else:
                    best_attribute_to_split = ''
                    comparison_value=1
                    for data_vertex in set(set(vertex['all_attributes'])-set([self.end_target])):

                        entropy_value2 = calculate_entropy_and_gini_index(self,vertex['set_data_data_range'],self.end_target,data_vertex)

                        if entropy_value2 < comparison_value:
                            bept2 = entropy_value2
                            best_attribute_to_split = data_vertex



                    if vertex['type'] != 'root_vertex':
                        if self.edge_entropy_gain:
                            if comparison_value >= entropy_value1:
                                vertex['type']='bottom'
                                continue



                    if ( self.types_of_attributes[best_attribute_to_split] == 'numeric' ):
                        split_result = self.generate_n_dimensions(self.n_tile, vertex['set_data_data_range'],best_attribute_to_split,self.end_target)
                        divided_list = set( split_result.keys() )
                    else:
                        divided_list = set( self.data[i][self.all_attributes[best_attribute_to_split]] for i in vertex['set_data_data_range'] )


                    vertex['column_selected']=best_attribute_to_split
                    td=vertex['child_vertexs']

                    for s in divided_list:

                        if ( self.types_of_attributes[best_attribute_to_split] == 'numeric' ):
                            set_of_rec = set( split_result[s]['bin_records'] )
                        else:
                            set_of_rec = set( i for i in vertex['set_data_data_range'] if self.data[i][self.all_attributes[best_attribute_to_split]] == s )

                        vall_attributes = vertex['all_attributes'].copy()
                        vall_attributes.pop(best_attribute_to_split)

                        childvertex_id = vertex_id+"--"+best_attribute_to_split+"--"+str(s)
                        td.append(childvertex_id)

                        childvertex = {'type': 'subTree', 'key': childvertex_id, 'all_attributes': vall_attributes,
                                       'set_data_data_range': set_of_rec, 'bin': s, 'child_vertexs': [],
                                       'parentAttrib': best_attribute_to_split, 'parentKey': vertex_id,
                                       'height': vertex['height'] + 1, 'column_selected': None}

                        self.list_of_vertex[childvertex_id]=childvertex
                        list_stack.append(childvertex_id )


    def prune(self,cross_validation_data):


        top_accuracy = self.calculate_classific_accuracy(cross_validation_data)
        vertex_list = copy.deepcopy(self.list_of_vertex)

        self.prune_the_entire_set(cross_validation_data)

        accuracy = self.calculate_classific_accuracy(cross_validation_data)
        if accuracy > top_accuracy:
            top_accuracy = accuracy
            vertex_list.clear()
            vertex_list = copy.deepcopy(self.list_of_vertex)

        while len(self.list_of_vertex.keys()) > 1:
            pruning_list_stack = {}

            for vertex_id in self.list_of_vertex:
                vertex = self.list_of_vertex[vertex_id]
                if vertex['type'] == 'bottom':
                    if vertex['height'] not in pruning_list_stack:
                        pruning_list_stack[vertex['height']] = []
                    pruning_list_stack[vertex['height']].append(vertex_id)


            visited = {}
            mincost_complexity_fact = float('inf')
            maximum_height_reached = max(pruning_list_stack.keys())
            prunevertex_id = None

            for height in reversed(xrange(1,maximum_height_reached+1)):

                while pruning_list_stack[height]:

                    vertex = self.list_of_vertex[pruning_list_stack[height].pop(0)]

                    if vertex['key'] in visited:
                        continue

                    visited[vertex['key']]=1

                    if vertex['parentKey']:
                        if vertex['height'] not in pruning_list_stack:
                            pruning_list_stack[vertex['height']] = []
                        pruning_list_stack[vertex['height']].append(vertex['parentKey'])


                    if vertex['type'] == 'bottom':
                        vertex['errors'] = vertex['error_for_majority']
                        vertex['bottom'] = 1
                        vertex['cost_complexity_fact'] = float('inf')

                    elif vertex['type'] != 'bottom':

                        class_error = 0.0
                        bottom =  0.0

                        for ck in vertex['child_vertexs']:
                            class_vertex = self.list_of_vertex[ck]
                            class_error +=  class_vertex['errors']
                            bottom += float(class_vertex['bottom'])

                        vertex['bottom'] = bottom
                        vertex['errors'] = class_error
                        if float(bottom) != 1:
                            vertex['cost_complexity_fact'] =  ( float(vertex['error_for_majority']) - class_error ) / float(bottom-1.0)
                        else:
                            vertex['cost_complexity_fact'] = float('-inf')

                        if vertex['cost_complexity_fact'] <= mincost_complexity_fact:
                            mincost_complexity_fact = vertex['cost_complexity_fact']
                            prunevertex_id = vertex['key']



            if prunevertex_id:
                self.prune_sub_list(prunevertex_id)


            accuracy = self.calculate_classific_accuracy(cross_validation_data)
            if accuracy > top_accuracy:
                top_accuracy = accuracy
                vertex_list.clear()
                vertex_list = copy.deepcopy(self.list_of_vertex)

        self.list_of_vertex.clear()
        self.list_of_vertex = vertex_list


    def prune_the_entire_set(self,cross_validation_data):


        top_accuracy = float('-inf')
        vertex_list = copy.deepcopy(self.list_of_vertex)
        savedlist_of_vertex = copy.deepcopy(self.list_of_vertex)

        for cost_complexity_fact in [0.4,0.6,0.8,1.0]:

            self.list_of_vertex.clear()
            self.list_of_vertex = copy.deepcopy(savedlist_of_vertex)

            pruning_list_stack = {}

            for vertex_id in self.list_of_vertex:
                vertex = self.list_of_vertex[vertex_id]
                if vertex['type'] == 'bottom':
                    if vertex['height'] not in pruning_list_stack:
                        pruning_list_stack[vertex['height']] = []
                    pruning_list_stack[vertex['height']].append(vertex_id)

            visited = {}

            maximum_height_reached = max(pruning_list_stack.keys())

            for height in reversed(xrange(1,maximum_height_reached+1)):

                while pruning_list_stack[height]:

                    vertex = self.list_of_vertex[pruning_list_stack[height].pop(0)]

                    if vertex['key'] in visited:
                        continue;

                    visited[vertex['key']]=1

                    if vertex['parentKey']:
                        if vertex['height'] not in pruning_list_stack:
                            pruning_list_stack[vertex['height']] = []
                        pruning_list_stack[vertex['height']].append(vertex['parentKey'])


                    if vertex['type'] == 'bottom':

                        vertex['errors'] = vertex['error_for_majority']
                        vertex['bottom'] = 1

                    if vertex['type'] != 'bottom':

                        class_error = 0
                        bottom = 0

                        for k in vertex['child_vertexs']:

                            class_vertex = self.list_of_vertex[k]
                            class_error +=  class_vertex['errors']
                            bottom += float(class_vertex['bottom'])

                        vertex['bottom'] = bottom
                        if vertex['error_for_majority']  > ( class_error + cost_complexity_fact*(bottom-1) ):
                            vertex['errors'] = class_error
                        else:
                            self.prune_sub_list(vertex['key'])

            accuracy = self.calculate_classific_accuracy(cross_validation_data)
            if accuracy > top_accuracy:
                top_accuracy = accuracy
                vertex_list.clear()
                vertex_list = copy.deepcopy(self.list_of_vertex)
                self.cost_complexity_fact = cost_complexity_fact

        self.list_of_vertex.clear()
        self.list_of_vertex = vertex_list


    def prune_sub_list(self,prunevertex_id):

        vertex = self.list_of_vertex[prunevertex_id]
        vertex['errors']= vertex['error_for_majority']
        vertex['type']='bottom'
        vertex['column_selected']=None
        vertex['bottom']=1
        vertex['cost_complexity_fact']=float('inf')

        prunelist_stack = vertex['child_vertexs']

        vertex['child_vertexs']=[]

        while prunelist_stack:

            prune_vertex_id = prunelist_stack.pop(0)
            prune_vertex = self.list_of_vertex[prune_vertex_id]

            if prune_vertex['type'] != 'bottom':
                for ck in prune_vertex['child_vertexs']:
                    prunelist_stack.append(ck)

            del self.list_of_vertex[prune_vertex['key']]

    def calculate_classific_accuracy(self,cross_validation_data):

        cross_valid_output = []
        for instance in cross_validation_data:
            result = self.classify( instance[:-1] )
            cross_valid_output.append( result == instance[-1])

        accuracy = float(cross_valid_output.count(True))/float(len(cross_valid_output))

        return accuracy


In [4]:
def calculate_entropy_and_gini_index(tree,set_data_data_range,parameter_result,decision_make_params):

    decision_list = {}

    r = tree.all_attributes[parameter_result]
    d = tree.all_attributes[decision_make_params]

    if ( tree.use_entropy and tree.types_of_attributes[decision_make_params] == 'numeric' ):

        split_result = tree.generate_n_dimensions(tree.n_tile, set_data_data_range,decision_make_params,parameter_result)

        divided_list = set( split_result.keys() )

        for s in divided_list:
            decision_list[s]={}
            decision_list[s]['count']=len(split_result[s]['bin_records'])

            for g in split_result[s]['bin_records_result_count'].keys():
                decision_list[s][g]=split_result[s]['bin_records_result_count'][g]


    else:
        for i in set_data_data_range:

            e = tree.data[i]

            if e[d] in decision_list:
                decision_list[e[d]]['count'] += 1
            else:
                decision_list[e[d]] = {}
                decision_list[e[d]]['count'] = 1

                if e[r] in decision_list[e[d]]:
                    decision_list[e[d]][e[r]] += 1
                else:
                    decision_list[e[d]][e[r]] = 1

    en=0.0

    if not tree.use_gini_index:

        for dc in decision_list:
            dct = decision_list[dc]['count']
            for rc in decision_list[dc]:
                if rc != 'count':
                    rct = decision_list[dc][rc]
                    if parameter_result == decision_make_params:
                        en += -(rct * (math.log(rct,2)-math.log(len(set_data_data_range),2)) )/len(set_data_data_range)
                    else:
                        en += -(dct*rct * (math.log(rct,2)-math.log(dct,2)) )/(dct*len(set_data_data_range))

    else:

        if ( parameter_result == decision_make_params ):

            en = 1.0

            for dc in decision_list:
                dct = decision_list[dc]['count']
                for rc in decision_list[dc]:
                    if rc != 'count':
                        rct = decision_list[dc][rc]
                        en = en-math.pow(float(rct)/float(len(set_data_data_range)),2)


        else:

            for dc in decision_list:
                dct = decision_list[dc]['count']
                en += float(dct)/float(tot)
                for rc in decision_list[dc]:
                    if rc != 'count':
                        rct = decision_list[dc][rc]
                        en = en-(float(dct)*math.pow(float(rct)/float(dct),2))/(tot)



    return en

In [5]:
def run_decision_tree():

    with open("/Users/b0203050/Downloads/wine-dataset.csv") as f:
        reader = csv.reader(f, delimiter=",")
        title = {e: i for i, e in enumerate(next(reader))}
        data = [map(convert_string_to_float,line) for line in reader]


    K = 10
    training_set = [x for i, x in enumerate(data) if i % K != 9]
    test_set = [x for i, x in enumerate(data) if i % K == 9]

    tree = DecisionTree()
    tree.learn_with_calculate_entropy_and_gini_index( training_set, title)


    set_of_results = []
    for instance in test_set:
        result = tree.classify( instance[:-1] )
        set_of_results.append( result == instance[-1])

    accuracy = float(set_of_results.count(True))/float(len(set_of_results))
    print "accuracy: %.4f" % accuracy

    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()

In [6]:
def convert_string_to_float(string):
    try:
        return float(string)
    except ValueError:
        return string

In [ ]:

if __name__ == "__main__":

    run_decision_tree()

training accuracy=0.8413
training accuracy=0.8345
training accuracy=0.8299
training accuracy=0.7914
training accuracy=0.8186
training accuracy=0.8073
training accuracy=0.8254
training accuracy=0.8073
training accuracy=0.8027
training accuracy=0.8318
training accuracy=0.8254
